In [ ]:
from keras.models import Model, load_model
from keras.layers import Input, Conv1D, Dense, Dropout, Lambda, Concatenate
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from wavenet_preprocessing import preprocessing
import datetime
import pandas as pd
import pathlib
import gc
from wavenet_model import*

In [ ]:
pages, cmp_enc_start, cmp_enc_end, pred_steps, series_array, \
data_start_date, data_end_date, train_pred_start, \
train_pred_end, train_enc_start, train_enc_end,  val_enc_start, \
val_enc_end, date_to_index, val_pred_start, val_pred_end = preprocessing()

gc.collect()

In [ ]:
# convolutional layer oparameters
n_filters = 128
filter_width = 5

model = build_model(n_filters, filter_width)
model.summary()

In [ ]:
batch_size = 2**5
epochs = 20

# sample of series from train_enc_start to train_enc_end
# sample of series from train_enc_start to train_enc_end

###ENCODER###
encoder_input_data = get_time_block_series(series_array, date_to_index,
                                        train_enc_start, train_enc_end)[:1000]


encoder_input_data, encode_series_mean, \
        encode_series_std = transform_series_encode(encoder_input_data)

np.isnan(encoder_input_data).any()


# sample of series from train_pred_start to train_pred_end

###DECODER###
decoder_target_data = get_time_block_series(series_array, date_to_index,
                                                train_pred_start, train_pred_end)[:1000]

decoder_target_data = transform_series_decode(decoder_target_data, \
                                              encode_series_mean, encode_series_std)

np.isnan(decoder_target_data).any()

# we append a lagged history of the target series to the input data,
# so that we can train with teacher forcing
lagged_target_history = decoder_target_data[:,:-1,:1]
encoder_input_data = np.concatenate([encoder_input_data, lagged_target_history], axis=1)

model.compile(Adam(), loss='mean_absolute_error')
history = model.fit(encoder_input_data, decoder_target_data,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=0.2)
# save the model
model.save(model_name + '.h5')

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend(['Train','Valid'])

In [ ]:
###ENCODER###
encoder_input_data = get_time_block_series(series_array,\
                                           date_to_index, val_enc_start, val_enc_end)

encoder_input_data, encode_series_mean,\
        encode_series_std =transform_series_encode(encoder_input_data)

###DECODER###
decoder_target_data = get_time_block_series(series_array, \
                                            date_to_index, val_pred_start, val_pred_end)

decoder_target_data = transform_series_decode(decoder_target_data, \
                                              encode_series_mean, encode_series_std)


predict_and_plot(encoder_input_data, 100, 2**8,decoder_target_data=decoder_target_data)
predict_and_plot(encoder_input_data, 70000, 2**8, decoder_target_data=decoder_target_data)

# In-sample prediction
in_sample_predicition = model.predict(encoder_input_data[700:701, :-24, :],2**8)

input = encoder_input_data[700:701, :, :].reshape(-1, 1)
in_sample_predicition = in_sample_predicition.reshape(-1, 1)

plt.figure()
plt.plot(input[-24:,:])
plt.plot(in_sample_predicition)
plt.legend(['data',"prediction"])

in_sample_predicition = model.predict(encoder_input_data[300:301, :-24, :],2**8)

input = encoder_input_data[300:301, :, :].reshape(-1, 1)
in_sample_predicition = in_sample_predicition.reshape(-1, 1)
plt.figure()
plt.plot(input[-24:,:])
plt.plot(in_sample_predicition)
plt.legend(['data',"prediction"])